In [3]:
# Import our module containing helper functions
import gravann

# Core imports
import numpy as np
import scipy
import pickle as pk
import os
from collections import deque
import torchquad as tquad
tquad.set_log_level("CRITICAL")
from copy import deepcopy

# pytorch
from torch import nn
import torch

# plotting stuff
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

C:\ProgramData\Miniconda3\envs\geodesynet\lib\site-packages\pyvista\themes.py:122: PyvistaDeprecationWarning: use "dark" instead of "night" theme
  warnings.warn('use "dark" instead of "night" theme', PyvistaDeprecationWarning)
18:57:19|TQ-INFO| Initializing torchquad.


Available devices  1
__pyTorch VERSION: 1.11.0
__CUDNN VERSION: 8200
__Number CUDA Devices: 1
Active CUDA Device: GPU 0
Setting default tensor type to Float32
Will use device  cuda:0


In [2]:
# Load the models to compare, on of 
# "eros", "bennu", "churyumov-gerasimenko", "itokawa", "planetesimal", "bennu_nu",

In [4]:
name = "itokawa_nu"

In [ ]:
# 1 - We load the masconCUBE
with open("models/"+name+"_masconCUBE.pk", "rb") as file:
    mascon_cube_points, mascon_cube_masses, mascon_cube_name = pk.load(file)
mascon_cube_points = torch.tensor(mascon_cube_points)
mascon_cube_masses = torch.tensor(mascon_cube_masses)

In [5]:
# 2 - We load the ground truth (also a mascon body)
with open("mascons/"+name+".pk", "rb") as file:
    mascon_points, mascon_masses, mascon_name = pk.load(file)
mascon_points = torch.tensor(mascon_points)
mascon_masses = torch.tensor(mascon_masses)

In [6]:
# 3 - We load the geodesyNET
encoding = gravann.direct_encoding()
model = gravann.init_network(encoding, n_neurons=100, model_type="siren", activation = gravann.AbsLayer())
model.load_state_dict(torch.load("models/"+name+".mdl"))

<All keys matched successfully>

In [9]:
results_mascon = gravann.validation_mascon(
    mascon_cube_points, mascon_cube_masses, mascon_points,mascon_masses, 
    asteroid_pk_path="3dmeshes/"+name+".pk", 
    N=10000, 
    batch_size=100, 
    progressbar=True)

NameError: name 'mascon_cube_points' is not defined

In [7]:
results_geodesyNet = gravann.validation(
    model, encoding, mascon_points, mascon_masses, 
    use_acc=True, 
    asteroid_pk_path="3dmeshes/"+name+".pk", 
    N=10000, 
    N_integration=300000,  # This needs to be the same as the number used during training, else precision will be lost
    batch_size=100, 
    progressbar=True)

C:\ProgramData\Miniconda3\envs\geodesynet\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Computing validation...:  62%|███████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                               | 48800/78672 [01:42<00:53, 557.48it/s]

Discarding 5977 of 16220 points in altitude sampler which did not meet requested altitude.


Computing validation...:  75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                          | 58800/78672 [02:37<01:17, 255.54it/s]

Discarding 10858 of 16220 points in altitude sampler which did not meet requested altitude.


Computing validation...:  87%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                     | 68800/78672 [03:30<00:38, 257.74it/s]

Discarding 14419 of 16220 points in altitude sampler which did not meet requested altitude.


Computing validation...: 78800it [04:20, 302.73it/s]                                                                                                                                                                                        


In [12]:
#results_geodesyNet.to_csv("models/"+name+"_validation_results.csv")

In [13]:
results_geodesyNet

,Altitude,Normalized L1 Loss,Normalized Relative Component Loss,RMSE,relRMSE
0,Low Altitude,0.036731,0.028634,0.598604,0.13881
1,High Altitude,0.007685,0.006218,0.238595,0.053619
2,Altitude_0,0.002238,0.002018,0.007902,0.00236
3,Altitude_1,0.00049,0.000529,0.00172,0.000613
4,Altitude_2,0.000049,0.000086,0.000174,0.000104


In [10]:
results_mascon

,Altitude,Normalized L1 Loss,Normalized Relative Component Loss,RMSE,relRMSE
0,Low Altitude,0.381709,0.467057,2.639638,1.356471
1,High Altitude,0.571023,0.849245,2.55243,1.940997
2,Altitude_0,0.002011,0.002718,0.007081,0.003323
3,Altitude_1,0.000498,0.000855,0.001756,0.001048
4,Altitude_2,0.000116,0.000322,0.000382,0.000381


In [14]:
factor =(7.329e10   * 6.67430e-11  / 352.1486930549145**2)

In [15]:
absolute = results_geodesyNet["Normalized L1 Loss"] * factor
relative = results_geodesyNet["Normalized Relative Component Loss"] 

In [16]:
# Prints the line for the table 1 in the paper
print(f"{absolute[2]:.2e} & {absolute[3]:.2e} & {absolute[4]:.2e} & {relative[2]*100:.2f} & {relative[3]*100:.2f} & {relative[4]*100:.3f}")

8.83e-08 & 1.93e-08 & 1.93e-09 & 0.20 & 0.05 & 0.009


In [14]:
masconCUBE = results_mascon["Normalized Relative Component Loss"]
geodesyNET = results_geodesyNet["Normalized Relative Component Loss"] 


In [15]:
print(f"{geodesyNET[0]*100:.2f} & {geodesyNET[1]*100:.2f}")
print(f"{masconCUBE[0]*100:.2f} & {masconCUBE[1]*100:.2f}")

27.46 & 47.11
46.71 & 84.92
